In [ ]:
## Cài đặt nếu thiếu thư viện để load model
# !pip install torch==2.6.0+cu118 torchaudio==2.6.0+cu118 torchvision==0.21.0+cu118 --index-url https://download.pytorch.org/whl/cu118
# !pip install transformers==4.51.3 xformers==0.0.27.post2 bitsandbytes==0.43.2
# !pip install unsloth==2025.6.12 unsloth-zoo==2025.6.8

In [ ]:
!pip install flask pyngrok

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Load base model 
base_model = AutoModelForCausalLM.from_pretrained(
    "unsloth/llama-3.2-3b-bnb-4bit",
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3.2-3b-bnb-4bit")

# Load LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, "dainlieu/Llama-3.2-3B-bnb-4bit-MedMCQA")
model.eval()

ngrok.set_auth_token("2zZ1SaL2XV6CLV1071XeNwrPaPH_66yx3VoYYuQSEZjepjZ1")
app = Flask(__name__)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.json

    question = data.get("question", "")
    choices = data.get("choices", [])

    if not question or not choices:
        return jsonify({"error": "Thiếu question hoặc choices"}), 400

    formatted_choices = "\n".join([f"{chr(65+i)}. {c}" for i, c in enumerate(choices)])
    prompt = (
        f"Question: {question}\n"
        f"Choices:\n{formatted_choices}\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=32, do_sample=False)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return jsonify({"prediction": result})

# Run ngrok và app Flask
public_url = ngrok.connect(5000)
print(f"Ngrok tunnel URL: {public_url}")
app.run(port=5000)